In [1]:
from utility import *
import argparse
import sys
from imgaug import augmenters as ima
import random

version of TF :  1.0.1


In [11]:
from shelve import open
def shelf(name, dic):
    shelf = open(name)
    shelf['images'] = dic['images']
    shelf['depths'] = dic['depths']
    shelf.close()

In [2]:
def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def _bytes_feature(value):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value=[value]))

In [76]:
def convert_to(d_data_sets, name):
    """Convert a data_set to tfrecords. """
    images = d_data_sets['images']
    dpt_images = d_data_sets['depths'] # depth images
    
    if name is not 'test':
        assert dpt_images.shape[0] == 795 #FLAGS.num_train
    else :
        assert dpt_images.shape[0] == 654 #FLAGS.num_test
    
    num_examples = images.shape[0]
    w = images.shape[2]
    h = images.shape[3]
    ch = images.shape[1]
    
    filename = os.path.join('./data', name + '.tfrecords') # dir path for writing
    print ("Writing ", filename)
    writer = tf.python_io.TFRecordWriter(filename) # make writer
    for index in range(num_examples):
        image_raw = images[index].tostring() # extract data to fill example protocol buffer
        depth_raw = dpt_images[index].tostring() 
        # Example protocol buffer
        example = tf.train.Example(features = tf.train.Features(feature={
                    'height' : _int64_feature(h),
                    'width' : _int64_feature(w),
                    'channel' : _int64_feature(ch),
                    'depth_raw' : _bytes_feature(depth_raw),
                    'image_raw' : _bytes_feature(image_raw)
                }))
        writer.write(example.SerializeToString()) # Serialize
    writer.close()

In [4]:
def divide_sets(data_sets):
    """divide data_sets into train and test sets."""
    train_sets={}
    test_sets={}
    
    train_idx=np.random.choice(1449, 795, replace=False)
    test_idx=[]
    for i in range(1449):
        if i not in train_idx:
            test_idx.append(i)
    train_idx = list(train_idx)
    train_idx.sort()
    test_idx.sort()
    
    train_sets['images'] = data_sets['images'][list(train_idx)]
    train_sets['depths'] = data_sets['depths'][list(train_idx)]
    test_sets['images'] = data_sets['images'][test_idx]
    test_sets['depths'] = data_sets['depths'][test_idx]
    
    return train_sets, test_sets

In [5]:
def img_process(imgs, dpts):
    def single(img, dpt):
        s = random.uniform(0.7, 1.3)
        tx = random.uniform(-40, 40); ty = random.uniform(-50, 50)
        if s>1.0:
            r = random.uniform(-30, 30)
        else:
            r = random.uniform(0,0)
        # scale
        aug = ima.Affine(scale=s)
        image = aug.augment_image(img); depth = aug.augment_image(dpt)
        depth = depth/s
        # translation
        aug2 = ima.Affine(translate_px={'x':int(tx), 'y':int(ty)})
        image = aug2.augment_image(image); depth = aug2.augment_image(depth)
        # rotation
        aug3 = ima.Affine(rotate=r)
        image = aug3.augment_image(image); depth = aug3.augment_image(depth)
        # crop
        #aug4 = ima.Crop(percent=(0.25), keep_size=False)
        #image = aug4.augment_image(image); depth = aug4.augment_image(depth)
        return image, depth
    
    raw_width = 640
    raw_height=480
    
    image, depth = single(imgs[0], dpts[0])
    images = np.reshape(image, (1, raw_height, raw_width, 3)) 
    depths = np.reshape(depth, (1, raw_height, raw_width, 1))
    for i in range(len(imgs) - 1):
        im, de = single(imgs[i+1], dpts[i+1])
        im = np.reshape(im, (1, raw_height, raw_width, 3)); de = np.reshape(de, (1, raw_height, raw_width, 1))
        images=np.concatenate((images, im), axis=0); depths=np.concatenate((depths, de), axis=0)
    
    return images, depths

In [6]:
data_sets = read_mat( './NYU_depthV2/nyu_depth_v2_labeled.mat')

In [7]:
# Divide data sets into train and test sets.
print ('dividing...')
train_sets, test_sets = divide_sets(data_sets)
del data_sets

dividing...


In [9]:
# converting test_sets, and raw_train sets
convert_to(test_sets, 'test')
convert_to(train_sets, 'train')
del test_sets